In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import mean_absolute_error, r2_score, root_mean_squared_error

file = Path(r"dG_RS.xlsx")
df = pd.read_excel(file, engine="openpyxl")

# Retrieve the labels of the independent variables
ligand_labels = df["ligand"].dropna().unique()
radical_labels = df["radical"].dropna().unique()
nucleophile_labels = df["nucleophile"].dropna().unique()

rs_root = 2020

In [95]:
fix_df = df.loc[((df["ligand"]=="L4") & (df["radical"]=="R16")) | ((df["radical"]=="R16") & (df["nucleophile"]=="N7")) | ((df["nucleophile"]=="N7") & (df["ligand"]=="L4")), :].copy(deep=True)
fix_df

,ligand,radical,nucleophile,dG
12,L6,R16,N7,-124.671077
36,L7,R16,N7,-145.577184
60,L14,R16,N7,-141.440641
84,L10,R16,N7,NaN
96,L4,R16,N10,-108.455604
102,L4,R16,N1,-138.685247
108,L4,R16,N7,-118.966389
109,L4,R5,N7,-115.656903
110,L4,R11,N7,-119.348542
111,L4,R1,N7,-116.355949


In [96]:
other_df = df.drop(fix_df.index)
other_df

,ligand,radical,nucleophile,dG
0,L6,R16,N10,-113.381554
1,L6,R5,N10,-113.262955
2,L6,R11,N10,-112.412052
3,L6,R1,N10,-114.731954
4,L6,R12,N10,-112.573949
...,...,...,...,...
115,L4,R5,N2,-135.495616
116,L4,R11,N2,-133.347024
117,L4,R1,N2,-135.684497
118,L4,R12,N2,-135.042554


In [97]:
# Populate the coefficient matrix and the right-hand side vector

def get_x_y(df):
    # Create the coefficient matrix and the right-hand side vector
    coefficient_matrix = []
    G = []
    for _, row in df.iterrows():
        if pd.notna(row["dG"]):
            i = np.where(ligand_labels == row["ligand"])[0][0]              # index of fixed ligand
            j = np.where(radical_labels == row["radical"])[0][0]            # index of fixed radical
            k = np.where(nucleophile_labels == row["nucleophile"])[0][0]    # index of fixed nucleophile
            coefficients = [0] * (
                len(ligand_labels) + len(radical_labels) + len(nucleophile_labels)
            )
            coefficients[i] = 1
            coefficients[len(ligand_labels) + j] = 1
            coefficients[len(ligand_labels) + len(radical_labels) + k] = 1
            coefficient_matrix.append(coefficients)
            G.append(row["dG"])

    coefficient_matrix = np.array(coefficient_matrix)
    G = np.array(G)
    G = -G
    return coefficient_matrix, G

In [98]:
# For the ternary linear equation G = L + R + N, there exists a scenario where constant values are assigned to the independent variables (L, R, N), allowing potential linear shifts in their absolute magnitudes.
# For instance, under transformations L' = L + 10, R' = R - 3, N' = N - 3, the equation G = L' + R' + N' still holds true. Different methodologies or initial guesses may lead to variations in these constant value assignments.
# Nevertheless, the relative values of LFER parameters within L, R, and N remain fundamentally consistent.
# Therefore, we adopt the strategy of fixing a single parameter value to obtain precise and stable absolute numerical solutions for L, R, and N.

def solve(matrix, values, fixed_x_val, fixed_y_val, fixed_z_val, mechanism):
    fixed_l_index = np.where(ligand_labels == "L4")[0][0]
    match mechanism:
        case "RE":
            fixed_r_index = len(ligand_labels) + np.where(radical_labels == "R11")[0][0]
            fixed_n_index = len(ligand_labels) + len(radical_labels) + np.where(nucleophile_labels == "N61")[0][0]
        case "RS":
            fixed_r_index = len(ligand_labels) + np.where(radical_labels == "R16")[0][0]
            fixed_n_index = len(ligand_labels) + len(radical_labels) + np.where(nucleophile_labels == "N7")[0][0]
        case "IP":
            fixed_r_index = len(ligand_labels) + np.where(radical_labels == "R94")[0][0]
            fixed_n_index = len(ligand_labels) + len(radical_labels) + np.where(nucleophile_labels == "N10")[0][0]
           
    fixed_indices = [fixed_l_index, fixed_r_index, fixed_n_index]
    fixed_values = [fixed_x_val, fixed_y_val, fixed_z_val]
    A_fixed = matrix[:, fixed_indices]
    adjusted_G = values - np.dot(A_fixed, fixed_values)

    free_columns_mask = np.ones(matrix.shape[1], dtype=bool)
    free_columns_mask[fixed_indices] = False
    A_free = matrix[:, free_columns_mask]

    try:
        params_free, _, _, _ = np.linalg.lstsq(A_free, adjusted_G, rcond=None)
    except np.linalg.LinAlgError:
        print("The matrix cannot be solved.")
        exit()

    solution = np.zeros(matrix.shape[1])
    solution[fixed_indices] = fixed_values
    solution[free_columns_mask] = params_free

    results = pd.DataFrame({
        "Conponent": [ligand_labels[i] for i in range(len(ligand_labels))]
                + [radical_labels[i] for i in range(len(radical_labels))]
                + [nucleophile_labels[i] for i in range(len(nucleophile_labels))],
        "Value": np.concatenate((
            solution[:len(ligand_labels)],
            solution[len(ligand_labels):len(ligand_labels)+len(radical_labels)],
            solution[-len(nucleophile_labels):]
        ))
    })

    return results

In [99]:
from sklearn.model_selection import KFold

mechanism = "RS"
fixed_l_val = 35.0543420075858          # L_RS of L4
fixed_r_val = 11.3134219163921          # R_RS of R16
fixed_n_val = 73.8959924869136          # N_RS of N7

rss = [rs_root * i for i in range(1, 11)]  # random states
fit_results: list[pd.DataFrame] = []
training_r2s = []
training_maes = []
training_rmses = []
test_r2s = []
test_maes = []
test_rmses = []
dG_results = []
idxs_1 = []
idxs_2 = []

for rs in rss:
    kf = KFold(n_splits=5, shuffle=True, random_state=rs)
    for fold, (train_idx, test_idx) in enumerate(kf.split(other_df)):

        train_df, test_df = other_df.iloc[train_idx, :], other_df.iloc[test_idx, :]
        combined_df = pd.concat([train_df, fix_df], axis=0)

        x_matrix, values = get_x_y(train_df)
        fit_result = solve(
            x_matrix, values, fixed_l_val, fixed_r_val, fixed_n_val, mechanism
        )

        mapping = fit_result.set_index("Conponent")["Value"].to_dict()

        # combine the fit result L, R, N values
        combined_df[["L", "R", "N"]] = combined_df[
            ["ligand", "radical", "nucleophile"]
        ]
        combined_df.loc[:, ["L", "R", "N"]] = combined_df.loc[:, ["L", "R", "N"]].replace(mapping)
        combined_df[["L", "R", "N"]] = combined_df[["L", "R", "N"]].astype(float)
        combined_df.loc[:, "dG_pred"] = combined_df.loc[:, ["L", "R", "N"]].sum(axis=1).mul(-1)

        test_df[["L", "R", "N"]] = test_df[
            ["ligand", "radical", "nucleophile"]
        ]
        test_df.loc[:, ["L", "R", "N"]] = test_df.loc[:, ["L", "R", "N"]].replace(mapping)
        test_df[["L", "R", "N"]] = test_df[["L", "R", "N"]].astype(float)
        test_df.loc[:, "dG_pred"] = test_df.loc[:, ["L", "R", "N"]].sum(axis=1).mul(-1)
        
        combined_notna_df = combined_df.dropna(axis=0, how="any")
        training_r2 = r2_score(combined_notna_df["dG"], combined_notna_df["dG_pred"])
        training_mae = mean_absolute_error(combined_notna_df["dG"], combined_notna_df["dG_pred"])
        training_rmse = root_mean_squared_error(combined_notna_df["dG"], combined_notna_df["dG_pred"])

        test_notna_df = test_df.dropna(axis=0, how="any")
        test_r2 = r2_score(test_notna_df["dG"], test_notna_df["dG_pred"])
        test_mae = mean_absolute_error(test_notna_df["dG"], test_notna_df["dG_pred"])
        test_rmse = root_mean_squared_error(test_notna_df["dG"], test_notna_df["dG_pred"])

        training_r2s.append(training_r2)
        training_maes.append(training_mae)
        training_rmses.append(training_rmse)
        test_r2s.append(test_r2)
        test_maes.append(test_mae)
        test_rmses.append(test_rmse)
        

        idxs_1.append(rs//rs_root)
        idxs_2.append(fold)

        fit_results.append(fit_result)

        # fit_result.to_csv(fr"RE_fit_results\solution_output_{mechanism}_round-{rs//1000}_fold-{fold}.csv", index=False)

performance_df = pd.DataFrame({"Round": idxs_1, "Fold": idxs_2, "Training R2": training_r2s, "Traing MAE": training_maes, "Training RMSE": training_rmses, "Test R2": test_r2s, "Test_MAE": test_maes, "Test_RMSE": test_rmses})
performance_df.dropna(axis=0, how="any").describe().round(3).loc["mean", "Test R2"]

/tmp/ipykernel_2119291/1630256011.py:38: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  combined_df.loc[:, ["L", "R", "N"]] = combined_df.loc[:, ["L", "R", "N"]].replace(mapping)
/tmp/ipykernel_2119291/1630256011.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df[["L", "R", "N"]] = test_df[
/tmp/ipykernel_2119291/1630256011.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentat

0.957

In [100]:
performance_df = pd.DataFrame({"Round": idxs_1, "Fold": idxs_2, "Training R2": training_r2s, "Traing MAE": training_maes, "Training RMSE": training_rmses, "Test R2": test_r2s, "Test_MAE": test_maes, "Test_RMSE": test_rmses})
performance_df

,Round,Fold,Training R2,Traing MAE,Training RMSE,Test R2,Test_MAE,Test_RMSE
0,1,0,0.984530,1.371404,1.802459,0.988127,1.704058,1.976783
1,1,1,0.987331,1.335059,1.776392,0.827801,2.850713,4.839869
2,1,2,0.986310,1.264114,1.664517,0.986529,1.835186,2.115248
3,1,3,0.987438,1.377189,1.727567,0.987902,1.262654,1.548284
4,1,4,0.990300,1.169669,1.549118,0.943697,2.525692,2.790189
5,2,0,0.986893,1.322839,1.780602,0.984611,1.317648,1.705900
6,2,1,0.988163,1.279891,1.695452,0.959562,2.101752,2.565883
7,2,2,0.987996,1.363290,1.697212,0.960264,1.618261,1.999176
8,2,3,0.990656,1.082353,1.402407,0.971547,2.333835,2.795627
9,2,4,0.984766,1.403060,1.785347,0.925811,2.785189,4.981254


In [101]:
r2s = []
maes = []
rmses = []
for l, d in performance_df.groupby("Round"):
    r2 = d.loc[:, "Test R2"].mean()
    mae = d.loc[:, "Test_MAE"].mean()
    rmse = d.loc[:, "Test_RMSE"].mean()
    r2s.append(r2)
    maes.append(mae)
    rmses.append(rmse)

print(np.array(r2s).mean())
print(np.array(maes).mean())
print(np.array(rmses).mean())
print(np.array(r2s).std())
print(np.array(maes).std())
print(np.array(rmses).std())

0.9573184849614236
1.8955066988402145
2.568101206939679
0.012058262279072666
0.09678170142825987
0.13008004445981403
